In [1]:
# ===============================================================
#  Library
# ===============================================================
import numpy as np
import polars as pl

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import warnings
warnings.simplefilter("ignore")

import lightgbm as lgb

import sys
sys.path.append("G:/マイドライブ/signate_MUFJ2023/")
from MUFJ.utils import get_score, seed_everything
from MUFJ.preprocessing import CustomOrdinalEncoder

from math import comb
import xgboost as xgb
from tqdm.auto import tqdm
from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# ===============================================================
#  CFG
# ===============================================================
class CFG:
    debug = False
    seed = 42
    n_splits = 5
    num_cores = 4
    data_dir = "G:/マイドライブ/signate_MUFJ2023/data/"
    stopping_rounds = 100
    save_dir = "G:/マイドライブ/signate_MUFJ2023/exp/"
    filename = "exp003"
    numerical_features = [
        "amount", 'cards_issued', 'credit_limit','year_pin_last_changed','current_age','retirement_age','birth_year','birth_month', 'latitude', 'longitude',
        'per_capita_income_zipcode', 'yearly_income_person', 'total_debt','fico_score', 'num_credit_cards', 'expires_month','expires_year','acct_open_date_month', 
        'acct_open_date_year', "YearsFromAcctOpenToPinChange",
        ]
    categorical_features = [
        "errors?", 'merchant_id', 'merchant_city','merchant_state','zip',"mcc",'use_chip','card_brand','card_type', 'has_chip','gender', 'city', 'state', 'zipcode',
        "card_id", "user_id", "same_zipcode_as_zip", "city_is_ONLINE",
        ]
    target_cols = ["is_fraud?"]

In [3]:
# ===============================================================
#  Utils
# ===============================================================
seed_everything(CFG.seed)

In [4]:
# ===============================================================
#  Data Loading
# ===============================================================
train = pl.read_csv(CFG.data_dir+"train.csv")
test = pl.read_csv(CFG.data_dir+"test.csv")
card = pl.read_csv(CFG.data_dir+"card.csv")
user = pl.read_csv(CFG.data_dir+"user.csv")

train = train.with_columns(
    pl.lit("train").alias("flag")
)
test = test.with_columns(
    [
        pl.lit(None, dtype=pl.Int64).alias("is_fraud?"),
        pl.lit("test").alias("flag"),
    ]
)

if CFG.debug:
    train = train.sample(n=10000, seed=CFG.seed)
    test = test.sample(n=1000, seed=CFG.seed)

all_data = pl.concat([train, test], how="align")
all_data = all_data.join(
    card, on=["user_id", "card_id"], how="left"
)
all_data = all_data.join(
    user, on="user_id", how="left"
)

In [5]:
# ===============================================================
#  Preprocessing
# ===============================================================
def preprocessing(all_data: pl.DataFrame) -> pl.DataFrame:
    
    all_data = all_data.with_columns(
        [   
            # str -> float
            pl.col("amount").apply(lambda x: x[1:]).cast(pl.Float64),
            pl.col("total_debt").apply(lambda x: x[1:]).cast(pl.Float64),
            pl.col("credit_limit").apply(lambda x: x[1:]).cast(pl.Float64),
            pl.col("yearly_income_person").apply(lambda x: x[1:]).cast(pl.Float64),
            pl.col("per_capita_income_zipcode").apply(lambda x: x[1:]).cast(pl.Float64),
            
            # str -> Datetime
            pl.col("expires").str.strptime(dtype=pl.Date, format="%m/%Y"),
            pl.col("acct_open_date").str.strptime(dtype=pl.Date, format="%m/%Y"),
            
            # bool
            (pl.col("zip") == pl.col("zipcode")).alias("same_zipcode_as_zip"),
            (pl.col("state") == pl.col("merchant_state")).alias("same_state"),
            #(pl.col("city") == pl.col("merchant_city")).alias("same_city"),
            (pl.col("merchant_city") == "ONLINE").alias("city_is_ONLINE"),
            #pl.when((pl.col("merchant_city").is_null())&(pl.col("merchant_city") != "ONLINE")) ## TODO: 上手くまとめられないかな
            #.then(pl.lit(True))
            #.otherwise(pl.lit(False))
            #.alias("city_is_not_America"),

            # user_id + card_id
            (pl.col("user_id").cast(pl.Utf8) + "-" + pl.col("card_id").cast(pl.Utf8)).alias("user_card_id"),
        ]
    )
    
    
    all_data = all_data.with_columns(
        [
            # Datetime -> Month, Year
            pl.col("expires").dt.year().suffix("_year"),
            pl.col("expires").dt.month().suffix("_month"),
            pl.col("acct_open_date").dt.year().suffix("_year"),
            pl.col("acct_open_date").dt.month().suffix("_month"),
        
            # feature_engineering
            #(pl.col("amount") - pl.col("credit_limit")).cast(pl.Float64).alias("remaining_credit"),
        ]
    )
    
    all_data = all_data.with_columns(
        [
            #(2023 - pl.col('year_pin_last_changed')).alias("YearsSincePinChange"),
            (pl.col("year_pin_last_changed") - pl.col("acct_open_date_year")).alias("YearsFromAcctOpenToPinChange"),
            #(pl.col("retirement_age") - pl.col("current_age")).alias("YearsUntilRetirement"),
            (pl.col("expires_year") - pl.col("year_pin_last_changed")).alias("YearsFromPinChangeToExpires"),
        ]
    )
    
    
    return all_data
all_data = preprocessing(all_data)

In [6]:
all_data[["amount", "mcc", "is_fraud?"]].filter(
    pl.col("is_fraud?") == 1
).groupby("mcc").agg(
    pl.col("amount").mean()
)

mcc,amount
i64,f64
7832,99.84344
5192,72.001414
5712,104.569021
4112,128.757472
4784,32.57881
3256,104.427245
5816,139.30034
3000,146.455369
3008,225.053538


In [7]:
all_data.columns

['index',
 'user_id',
 'card_id',
 'amount',
 'errors?',
 'is_fraud?',
 'merchant_id',
 'merchant_city',
 'merchant_state',
 'zip',
 'mcc',
 'use_chip',
 'flag',
 'card_brand',
 'card_type',
 'expires',
 'has_chip',
 'cards_issued',
 'credit_limit',
 'acct_open_date',
 'year_pin_last_changed',
 'current_age',
 'retirement_age',
 'birth_year',
 'birth_month',
 'gender',
 'address',
 'city',
 'state',
 'zipcode',
 'latitude',
 'longitude',
 'per_capita_income_zipcode',
 'yearly_income_person',
 'total_debt',
 'fico_score',
 'num_credit_cards',
 'same_zipcode_as_zip',
 'same_state',
 'city_is_ONLINE',
 'user_card_id',
 'expires_year',
 'expires_month',
 'acct_open_date_year',
 'acct_open_date_month',
 'YearsFromAcctOpenToPinChange',
 'YearsFromPinChangeToExpires']

In [8]:
df = all_data.filter(
    pl.col("flag") == "train"
).groupby("user_id").agg(
    [
        pl.col("is_fraud?").mean().alias("mean"),
        pl.col("is_fraud?").count().alias("counts")
    ]
)
df = df.with_columns(
    [
        (pl.col("mean") * pl.col("counts")).alias("number")
    ]
)
df

user_id,mean,counts,number
i64,f64,u32,f64
80,0.077764,4115,320.0
1328,0.070259,7999,562.0
640,0.058388,4453,260.0
920,0.090909,4301,391.0
1592,0.045191,7922,358.0
1032,0.127886,6193,792.0
848,0.079856,4984,398.0
1400,0.1041,2805,292.0
720,0.094446,4627,437.0


In [9]:
df["number"].value_counts()

number,counts
f64,u32
403.0,1
548.0,1
650.0,1
324.0,1
416.0,2
236.0,1
336.0,1
53.0,1
141.0,1


In [10]:
all_data.filter(
    pl.col("flag") == "train"
).groupby("user_id").agg(
    pl.col("is_fraud?").mean(),
)

user_id,is_fraud?
i64,f64
1086,0.105031
1789,0.051026
1947,0.05818
1385,0.065902
591,0.084906
1342,0.098217
1526,0.063611
371,0.025566
1117,0.062604


In [11]:
all_data.filter(
   ( pl.col("user_id") == 39)&(pl.col("flag") == "train")
)["current_age"].value_counts()

current_age,counts
i64,u32
57,2894


In [12]:
all_data.filter(
   ( pl.col("user_id") == 39)&(pl.col("flag") == "test")
)["current_age"].value_counts()

current_age,counts
i64,u32
57,2861


In [13]:
all_data.filter(
    pl.col("user_id") == 39
).describe()

describe,index,user_id,card_id,amount,errors?,is_fraud?,merchant_id,merchant_city,merchant_state,zip,mcc,use_chip,flag,card_brand,card_type,expires,has_chip,cards_issued,credit_limit,acct_open_date,year_pin_last_changed,current_age,retirement_age,birth_year,birth_month,gender,address,city,state,zipcode,latitude,longitude,per_capita_income_zipcode,yearly_income_person,total_debt,fico_score,num_credit_cards,same_zipcode_as_zip,same_state,city_is_ONLINE,user_card_id,expires_year,expires_month,acct_open_date_year,acct_open_date_month,YearsFromAcctOpenToPinChange,YearsFromPinChangeToExpires
str,f64,f64,f64,f64,str,f64,f64,str,str,f64,f64,str,str,str,str,str,str,f64,f64,str,f64,f64,f64,f64,f64,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64
"""count""",5755.0,5755.0,5755.0,5755.0,"""5755""",5755.0,5755.0,"""5755""","""5755""",5755.0,5755.0,"""5755""","""5755""","""5755""","""5755""","""5755""","""5755""",5755.0,5755.0,"""5755""",5755.0,5755.0,5755.0,5755.0,5755.0,"""5755""","""5755""","""5755""","""5755""",5755.0,5755.0,5755.0,5755.0,5755.0,5755.0,5755.0,5755.0,5755.0,5755.0,5755.0,"""5755""",5755.0,5755.0,5755.0,5755.0,5755.0,5755.0
"""null_count""",0.0,0.0,0.0,0.0,"""0""",2861.0,0.0,"""0""","""650""",1030.0,0.0,"""0""","""0""","""0""","""0""","""0""","""0""",0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1030.0,650.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",509603.297133,39.0,0.27437,147.014868,null,0.042847,205976.209209,null,null,6357.121481,5697.40695,null,null,null,null,null,null,1.27437,13902.205387,null,2011.371851,57.0,64.0,1962.0,12.0,null,null,null,null,2048.0,42.02,-71.21,37407.0,76274.0,102611.0,698.0,2.0,0.724444,0.826641,0.112945,null,2021.82311,9.54874,2009.194961,6.371851,2.17689,10.45126
"""std""",255616.073592,0.0,0.446235,163.145806,null,0.202548,110428.760604,null,null,16253.820087,1107.056551,null,null,null,null,null,null,0.446235,5228.536536,null,2.231175,0.0,0.0,0.0,0.0,null,null,null,null,0.0,3.6951e-13,7.2482e-13,0.0,0.0,0.0,0.0,0.0,0.446841,0.378595,0.316553,null,1.338705,0.89247,3.569881,2.231175,1.338705,0.89247
"""min""",94062.0,39.0,0.0,0.072,"""ERROR""",0.0,613.0,"""Alanson""","""CA""",1030.0,3000.0,"""Chip Transacti…","""test""","""Amex""","""Credit""","""2021-09-01""","""YES""",1.0,5400.0,"""2007-05-01""",2010.0,57.0,64.0,1962.0,12.0,"""Female""","""442 Burns Boul…","""Mansfield""","""MA""",2048.0,42.02,-71.21,37407.0,76274.0,102611.0,698.0,2.0,0.0,0.0,0.0,"""39-0""",2021.0,9.0,2007.0,5.0,0.0,9.0
"""25%""",279964.0,39.0,0.0,53.607,null,0.0,110602.0,null,null,2048.0,5300.0,null,null,null,null,null,null,1.0,5400.0,null,2010.0,57.0,64.0,1962.0,12.0,null,null,null,null,2048.0,42.02,-71.21,37407.0,76274.0,102611.0,698.0,2.0,null,null,null,null,2021.0,9.0,2007.0,5.0,0.0,9.0
"""50%""",469411.0,39.0,0.0,90.686,null,0.0,202459.0,null,null,2048.0,5541.0,null,null,null,null,null,null,1.0,17117.0,null,2010.0,57.0,64.0,1962.0,12.0,null,null,null,null,2048.0,42.02,-71.21,37407.0,76274.0,102611.0,698.0,2.0,null,null,null,null,2021.0,9.0,2007.0,5.0,3.0,11.0
"""75%""",737957.0,39.0,1.0,183.3,null,0.0,304208.0,null,null,2050.0,5912.0,null,null,null,null,null,null,2.0,17117.0,null,2015.0,57.0,64.0,1962.0,12.0,null,null,null,null,2048.0,42.02,-71.21,37407.0,76274.0,102611.0,698.0,2.0,null,null,null,null,2024.0,11.0,2015.0,10.0,3.0,11.0
"""max""",929235.0,39.0,1.0,1794.999,"""OK""",1.0,406662.0,"""Ypsilanti""","""WA""",98422.0,9402.0,"""Swipe Transact…","""train""","""Visa""","""Debit""","""2024-11-01""","""YES""",2.0,17117.0,"""2015-10-01""",2015.0,57.0,64.0,1962.0,12.0,"""Female""","""442 Burns Boul…","""Mansfield""","""MA""",2048.0,42.02,-71.21,37407.0,76274.0,102611.0,698.0,2.0,1.0,1.0,1.0,"""39-1""",2024.0,11.0,2015.0,10.0,3.0,11.0


In [14]:
all_data.filter(
    pl.col("user_id") == 39
)["card_id"].value_counts()

card_id,counts
i64,u32
0,4176
1,1579


In [15]:
all_data.filter(
    pl.col("user_id") == 39
)["is_fraud?"].value_counts()

is_fraud?,counts
i64,u32
0,2770
1,124
null,2861


In [16]:
[col for col in all_data.columns if len(all_data.filter(pl.col("user_id") == 39)[col].unique()) == 1]

['user_id',
 'has_chip',
 'current_age',
 'retirement_age',
 'birth_year',
 'birth_month',
 'gender',
 'address',
 'city',
 'state',
 'zipcode',
 'latitude',
 'longitude',
 'per_capita_income_zipcode',
 'yearly_income_person',
 'total_debt',
 'fico_score',
 'num_credit_cards']

In [17]:
[col for col in all_data.columns if len(all_data.filter(pl.col("user_id") == 39)[col].unique()) != 1]

['index',
 'card_id',
 'amount',
 'errors?',
 'is_fraud?',
 'merchant_id',
 'merchant_city',
 'merchant_state',
 'zip',
 'mcc',
 'use_chip',
 'flag',
 'card_brand',
 'card_type',
 'expires',
 'cards_issued',
 'credit_limit',
 'acct_open_date',
 'year_pin_last_changed',
 'same_zipcode_as_zip',
 'same_state',
 'city_is_ONLINE',
 'user_card_id',
 'expires_year',
 'expires_month',
 'acct_open_date_year',
 'acct_open_date_month',
 'YearsFromAcctOpenToPinChange',
 'YearsFromPinChangeToExpires']

In [18]:
all_data.groupby(["user_id", "cards_issued"]).agg(
    pl.col("is_fraud?").mean()
)

user_id,cards_issued,is_fraud?
i64,i64,f64
1149,1,0.088686
306,2,0.044953
783,2,0.05289
711,2,0.020508
1947,1,0.090841
1986,2,0.102998
821,1,0.050053
1032,1,0.12013
1097,2,0.083748


In [19]:
all_data.with_columns(
    pl.col("*").n_unique()
)

index,user_id,card_id,amount,errors?,is_fraud?,merchant_id,merchant_city,merchant_state,zip,mcc,use_chip,flag,card_brand,card_type,expires,has_chip,cards_issued,credit_limit,acct_open_date,year_pin_last_changed,current_age,retirement_age,birth_year,birth_month,gender,address,city,state,zipcode,latitude,longitude,per_capita_income_zipcode,yearly_income_person,total_debt,fico_score,num_credit_cards,same_zipcode_as_zip,same_state,city_is_ONLINE,user_card_id,expires_year,expires_month,acct_open_date_year,acct_open_date_month,YearsFromAcctOpenToPinChange,YearsFromPinChangeToExpires
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
929241,97,9,185744,2,3,31533,4970,129,9117,109,3,2,4,3,119,2,3,350,182,18,46,18,46,12,2,97,94,37,96,92,92,94,97,94,76,9,3,3,2,413,21,12,28,12,20,29
929241,97,9,185744,2,3,31533,4970,129,9117,109,3,2,4,3,119,2,3,350,182,18,46,18,46,12,2,97,94,37,96,92,92,94,97,94,76,9,3,3,2,413,21,12,28,12,20,29
929241,97,9,185744,2,3,31533,4970,129,9117,109,3,2,4,3,119,2,3,350,182,18,46,18,46,12,2,97,94,37,96,92,92,94,97,94,76,9,3,3,2,413,21,12,28,12,20,29
929241,97,9,185744,2,3,31533,4970,129,9117,109,3,2,4,3,119,2,3,350,182,18,46,18,46,12,2,97,94,37,96,92,92,94,97,94,76,9,3,3,2,413,21,12,28,12,20,29
929241,97,9,185744,2,3,31533,4970,129,9117,109,3,2,4,3,119,2,3,350,182,18,46,18,46,12,2,97,94,37,96,92,92,94,97,94,76,9,3,3,2,413,21,12,28,12,20,29
929241,97,9,185744,2,3,31533,4970,129,9117,109,3,2,4,3,119,2,3,350,182,18,46,18,46,12,2,97,94,37,96,92,92,94,97,94,76,9,3,3,2,413,21,12,28,12,20,29
929241,97,9,185744,2,3,31533,4970,129,9117,109,3,2,4,3,119,2,3,350,182,18,46,18,46,12,2,97,94,37,96,92,92,94,97,94,76,9,3,3,2,413,21,12,28,12,20,29
929241,97,9,185744,2,3,31533,4970,129,9117,109,3,2,4,3,119,2,3,350,182,18,46,18,46,12,2,97,94,37,96,92,92,94,97,94,76,9,3,3,2,413,21,12,28,12,20,29
929241,97,9,185744,2,3,31533,4970,129,9117,109,3,2,4,3,119,2,3,350,182,18,46,18,46,12,2,97,94,37,96,92,92,94,97,94,76,9,3,3,2,413,21,12,28,12,20,29
